Load Stereo-seq data of spleen from Mouse 2 (MML)

In [1]:
import stereo as st
import scanpy as sc
import utils_stereoseq as us
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import squidpy as sq

def bin_trascripts_barcodes(bin_size, gem_path_m4, gef_path_m4, bc_path_m4, out_path):
    print(f"binning transcripts and barcodes {bin_size}")
    gem = st.io.read_gem(file_path=gem_path_m4, bin_size=bin_size)

    gem.tl.raw_checkpoint()
    adata = st.io.stereo_to_anndata(data=gem, flavor="scanpy")
    adata.uns["attr"] = gem.attr

    bc_data = pd.read_csv(bc_path_m4, sep="\t")
    bc_data = bc_data.rename(columns={"gene": "barcode"})

    # returns table with "barcode", "count_binned", "bin_x", "bin_y", "cell_id", "x_center", "y_center"
    bc_data = us.bin_barcodes(
        bc_data,
        minX=adata.uns["attr"]["minX"],
        minY=adata.uns["attr"]["minY"],
        gef_raw_path=gef_path_m4,
        bin_size=bin_size
    )
    
    # already check if spot is in adata, for QC plot
    bc_data["isin_adata"] = bc_data["cell_id"].isin(adata.obs.index.values)
    
    # save barcode data with all barcodes per bin, and all bins
    bc_data.to_csv(f"{out_path}/mouse4_bin{bin_size}_bc_counts.tsv", sep="\t")
    
    # only keep most abundant barcode per bin
    bc_data = bc_data.sort_values("count_binned", ascending=False).groupby('cell_id').head(1)

    # create column to merge on
    adata.obs["cell_id"] = adata.obs.index.values

    # merge
    adata.obs = adata.obs.reset_index().merge(
        bc_data[["cell_id", "barcode", "count_binned"]], 
        on="cell_id", 
        how="left"
    ).set_index('index')


    sc.pp.calculate_qc_metrics(adata, log1p=False, inplace=True, use_raw=True)

    # save joint data
    print(f"writing output {bin_size}")
    adata.write(filename=f"{out_path}/mouse4_bin{bin_size}_bc.h5ad")

    # export barcode counts on tissue section for bar plot
    adata.obs["barcode"][~adata.obs["barcode"].isna()].value_counts().to_csv(
        f"{out_path}/mouse4_bin{bin_size}_bc_counts_top1.tsv", sep="\t"
    )
    return(adata)


####################################

def filter_cluster(adata, bin_size, out_path, min_counts, max_counts = False):
    print(f"filter cells {bin_size}")
    sc.pp.filter_cells(adata, min_counts=min_counts)
    if max_counts:
        sc.pp.filter_cells(adata, max_counts=max_counts)

    sc.pp.log1p(adata)
    sc.pp.highly_variable_genes(adata)
    adata.raw = adata

    print(f"scale {bin_size}")
    if bin_size <= 10:
        # uses too much memory to zero center
        sc.pp.scale(adata, zero_center=False)
    else:
        sc.pp.scale(adata)

    print(f"pca {bin_size}")
    sc.tl.pca(adata, svd_solver='arpack')

    sc.pp.neighbors(adata)

    print(f"umap {bin_size}")
    sc.tl.umap(adata)
    
    print(f"leiden {bin_size}")
    sc.tl.leiden(adata, resolution=1, key_added="leiden_1")
    sc.tl.leiden(adata, resolution=0.7, key_added="leiden_0.7")
    sc.tl.leiden(adata, resolution=0.5, key_added="leiden_0.5")
    sc.tl.leiden(adata, resolution=0.2, key_added="leiden_0.2")

    print(f"DEG {bin_size}")
    sc.tl.rank_genes_groups(adata, 'leiden_0.7', method='t-test', use_raw=True)

    # save adata
    adata.write(filename=f"{out_path}/mouse4_bin{bin_size}_bc_clustered.h5ad")

/data/projects/punim0613/JiaDong/miniconda3/envs/stereo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gem_path_m4 = "/data/gpfs/projects/punim0613/JiaDong/Dawson/MouseSpleen/Mouse2/SS200000334TR_A2.lasso.mouse2_whole_tissue.gem.gz"
gef_path_m4 = "/data/gpfs/projects/punim0613/JiaDong/Dawson/MouseSpleen/Mouse2/SS200000334TR_A2.gef"
bc_path_m4 = "/data/gpfs/projects/punim0613/JiaDong/Dawson/MouseSpleen/Mouse2/DP8400028335BL_L01_read_1.unmapped_reads.counts.tsv"

out_path = "/data/gpfs/projects/punim0613/JiaDong/Dawson/MouseSpleen/Mouse2"

In [24]:
bin_size = 50
gem = st.io.read_gef(file_path=gef_path_m4, bin_size=bin_size)

[2024-10-22 09:05:11][Stereo][68776][MainThread][23346822308864][reader][1194][INFO]: read_gef begin ...


[2024-10-22 09:05:21][Stereo][68776][MainThread][23346822308864][reader][1369][INFO]: the matrix has 168998 cells, and 18527 genes.
[2024-10-22 09:05:21][Stereo][68776][MainThread][23346822308864][reader][1370][INFO]: read_gef end.


In [30]:
gem.tl.raw_checkpoint()
adata = st.io.stereo_to_anndata(data=gem, flavor="scanpy")
adata.uns["attr"] = gem.attr

[2024-10-22 09:06:45][Stereo][68776][MainThread][23346822308864][reader][914][INFO]: Adding sample in adata.obs['orig.ident'].
[2024-10-22 09:06:45][Stereo][68776][MainThread][23346822308864][reader][917][INFO]: Adding data.position as adata.obsm['spatial'] .
[2024-10-22 09:06:45][Stereo][68776][MainThread][23346822308864][reader][922][INFO]: Adding data.position as adata.obs['x'] and adata.obs['y'] .
[2024-10-22 09:06:45][Stereo][68776][MainThread][23346822308864][reader][1062][INFO]: Adding data.tl.raw.exp_matrix as adata.raw .
[2024-10-22 09:06:46][Stereo][68776][MainThread][23346822308864][reader][1084][INFO]: Finished conversion to anndata.


In [11]:
bc_data = pd.read_csv(bc_path_m4, sep="\t")
bc_data = bc_data.rename(columns={"gene": "barcode"})
# returns table with "barcode", "count_binned", "bin_x", "bin_y", "cell_id", "x_center", "y_center"
bc_data = us.bin_barcodes(
    bc_data,
    minX=adata.uns["attr"]["minX"],
    minY=adata.uns["attr"]["minY"],
    gef_raw_path=gef_path_m4,
    bin_size=bin_size
)

[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1398][INFO]: This is GEF file which contains traditional bin infomation.
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1399][INFO]: bin_type: bins
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1402][INFO]: Bin size list: ['bin1', 'bin10', 'bin100', 'bin20', 'bin200', 'bin50', 'bin500']
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1408][INFO]: Resolution: 500
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1411][INFO]: Gene count: 18527
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1420][INFO]: offsetX: 0
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1423][INFO]: offsetY: 2
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1426][INFO]: Width: 26456
[2024-10-21 23:26:39][Stereo][68776][MainThread][23346822308864][reader][1429][INFO]: H

,orig.ident,x,y
50,sample,0,50
2500,sample,0,2500
4400,sample,0,4400
5550,sample,0,5550
6200,sample,0,6200
...,...,...,...
113601884994600,sample,26450,15400
113601884995500,sample,26450,16300
113601884996250,sample,26450,17050
113601885002650,sample,26450,23450


In [33]:
# already check if spot is in adata, for QC plot
bc_data["isin_adata"] = bc_data["cell_id"].isin(adata.obs.index.values)

# save barcode data with all barcodes per bin, and all bins
bc_data.to_csv(f"{out_path}/mouse2_bin{bin_size}_bc_counts.tsv", sep="\t")

# only keep most abundant barcode per bin
bc_data = bc_data.sort_values("count_binned", ascending=False).groupby('cell_id').head(1)

# create column to merge on
adata.obs["cell_id"] = adata.obs.index.values

# merge
adata.obs = adata.obs.reset_index().merge(
    bc_data[["cell_id", "barcode", "count_binned"]], 
    on="cell_id", 
    how="left"
).set_index('index')


sc.pp.calculate_qc_metrics(adata, log1p=False, inplace=True, use_raw=True)

adata.X = adata.X.astype(np.float32)

# save joint data
print(f"writing output {bin_size}")
adata.write(filename=f"{out_path}/mouse2_bin{bin_size}_bc.h5ad")

[2024-10-18 17:06:32][Stereo][283539][MainThread][22799946028032][reader][157][INFO]: the martrix has 25108 cells, and 18109 genes.
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][914][INFO]: Adding sample in adata.obs['orig.ident'].
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][917][INFO]: Adding data.position as adata.obsm['spatial'] .
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][922][INFO]: Adding data.position as adata.obs['x'] and adata.obs['y'] .
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][1062][INFO]: Adding data.tl.raw.exp_matrix as adata.raw .
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][1084][INFO]: Finished conversion to anndata.
[2024-10-18 17:06:33][Stereo][283539][MainThread][22799946028032][reader][1398][INFO]: This is GEF file which contains traditional bin infomation.
[2024-10-18 17:06:33][Stereo][283539][MainThread][227999460280

writing output 50


TypeError: Can't implicitly convert non-string objects to strings

Above error raised while writing key 'barcode' of <class 'h5py._hl.group.Group'> to /

In [19]:
adata.

Index(['25555055426150', '25769803790800', '25769803791700', '25984552155750',
       '25984552155950', '25984552156450', '25984552156500', '26199300520500',
       '26199300521250', '26199300521600',
       ...
       '73014444045300', '25984552155000', '45526653355350', '73229192410650',
       '45097156618650', '67001489828550', '33930241651000', '32212254732900',
       '66571993098850', '42090679512850'],
      dtype='object', length=25108)

In [25]:
import scipy as sp

In [29]:
sp.sparse.save_npz(f"{out_path}/mouse2_bin{bin_size}.npz", adata.X.astype(np.float32))